In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import scipy
import numpy as np
import pandas as pd
import os

import os
os.chdir('..')

from calcium import calcium
from wheel import wheel
from visualize import visualize
from tqdm import trange

from scipy.io import loadmat

import umap

from sklearn.decomposition import PCA
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


#
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


#
from tsnecuda import TSNE

# 
np.set_printoptions(suppress=True)


Autosaving every 180 seconds


In [25]:
####################################
######### BINARIZE DATA ############
###################################

scale = 100

# root_dir = '/media/cat/4TB/donato/laurenz/'
# animal_id = 'DON-005099'
# session = '20210609'

# root_dir = '/media/cat/4TB/donato/steffen/'
# animal_id = 'DON-004366'
# session = '20210228'

# root_dir = '/media/cat/4TB/donato/'
# animal_id = 'DON-006084'
# session = '20210519'

root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-003343'
session = 'DON-003343_20210222'


################################################## 
##################################################
##################################################
session_dir = os.path.join(root_dir, 
                           animal_id, 
                           session, 'suite2p','plane0')

#
c = calcium.Calcium()
c.verbose = True
c.recompute = True
c.root_dir = root_dir
c.animal_id = animal_id
c.session = session
c.load_suite2p()

# Oasis/spike parameters
c.oasis_thresh_prefilter = 5                       # min oasis spike value that survives 
c.min_thresh_std_oasis = .02                          # upphase binarizatino step: min std for binarization of smoothed oasis curves
c.min_width_event_oasis = 2                              # <--- min width of the window in frame times
c.min_event_amplitude = 1                           # oasis scaled spikes: float point scaling boolean events; minimum amplitude required (removes very small amplitude events)

# Fluorescence parameters
c.min_thresh_std_Fluorescence_onphase = 1.5         # onphase binarization step: min x std for binarization of Fluorescence events
c.min_thresh_std_Fluorescence_upphase = 1.5         # upphase binarization step: min x std for binarization of Fluorescence events
c.min_width_event_Fluorescence = 15                              # <--- min width of the window in frame times


# select a cell
selected_cell = np.arange(c.F.shape[0])
selected_cell = 0
c.cell_name = animal_id+" " +session + " cell: "+ str(selected_cell)

c.load_binarization()

# if c.recompute==True:
#     c.F = c.F[selected_cell][None]
#     c.F_filtered = c.F_filtered[selected_cell][None]
#     c.spks = c.spks[selected_cell][None]
#     selected_cell = 0

#     # run binarization
#     c.binarize_fluorescence()


    
##################################################
############### START PLOTTING ###################
##################################################
fig = plt.figure()
plt.style.use("dark_background")


##################################################
############ PLOT RAW FLUORESCENCE ###############
##################################################
c.plot_traces(c.F - np.median(c.F,axis=1)[None].T, 
              [selected_cell], 
              'raw F',
              'cyan',
              .3,
              1
                 )

####################################################
########## PLOTFILTERED FLUORESCENCE ###############
####################################################
if True:
    c.plot_traces(c.F_filtered, 
                  [selected_cell], 
                  'filtered F ',
                  'cyan',
                  1,
                  2
                 )

####################################################
######## PLOT BINARIZED FLUORESCENCE ###############
####################################################
if True:
    c.plot_traces(c.F_onphase_bin*40,
                  [selected_cell], 'Fluoresence ONPHASE',
                 'red',
                  0.5,
                  6)
    # 
    c.plot_traces(c.F_onphase_bin*50,
              [selected_cell], 'Fluorescence UPPPHASE',
             'mistyrose',
              0.5,
              6)


#################################################
############### PLOT RAW OASIS ##################
#################################################
if True:
    c.plot_traces(c.spks*1,
              [selected_cell],
              'spikes (excluding prob < '+str(c.oasis_thresh_prefilter)+ ")",
              'white',
              .9,
              5
    )


#################################################
####### PLOT OASIS X FLUORESCENCE ###############
#################################################
if True:
    c.plot_traces(c.spks_x_F*.002,
                  [selected_cell], 
                  'spikes * Fluorescence',
                 'lightgreen',
                 1,
                 3)

###################################################################
########### PLOT BINARIZED + SCALED OASIS x FLUROESCENCE ##########
###################################################################
if True:
    c.plot_traces(c.spks_smooth_bin*.5,
                  [selected_cell], 
                  'binarized spikes (upphase)',
                  'gold',
                 .9,
                 6)

############################################ 
plt.legend(fontsize=20, ncol=3)
#plt.xlim(1480,1500)
plt.suptitle(c.cell_name)
plt.show()






        self.F (fluorescence):  (751, 55740)
         self.Fneu (neuropile):  (751, 55740)
         self.iscell (cell classifier output):  (965, 2)
         # of good cells:  (751,)
         self.ops:  ()
         self.spks (deconnvoved spikes):  (751, 55740)
         self.stat (footprints?):  (751,)
         mean std over all cells :  8.5


binarizing continuous traces: 100%|███████████████████████████████████████████████████████████| 751/751 [00:02<00:00, 323.05it/s]
/home/cat/code/manifolds/calcium/calcium.py:428: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  locs = np.array(locs)
binarizing continuous traces: 100%|████████████████████████████████████████████████████████████| 751/751 [00:10<00:00, 73.45it/s]
convolving oasis with exponentional and filtering: 100%|██████████████████████████████████████| 751/751 [00:04<00:00, 175.56it/s]
/home/cat/anaconda3/envs/manifolds/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
scaling binarized data: 100%|██████████████████████████████████

In [15]:
############################################
############################################
############################################
print (c.F.shape)
stds = np.std(c.F,axis=1)
print (stds.shape)
y = np.histogram(stds, bins = np.arange(0,100,2))

fig=plt.figure()
plt.plot(y[1][:-1], y[0])
plt.xlabel("Standard deviation of Fluorescence signal")
plt.ylabel("# of cells")
plt.suptitle(animal_id + " - session: "+session)
plt.show()

(882, 55740)
(882,)


In [141]:
###########################################################
###########################################################
###########################################################
data = c.F[0]
print (data)

width = 5
bins = np.arange(-1000,1000,width)
#
y = np.histogram(data, bins)

argmax = np.argmax(y[0])
data = data-(y[1][argmax])


mean = np.mean(data)
#data = data-mean
std = np.std(data)



fig = plt.figure()
plt.plot([std,std],[0,1],'--')
plt.plot([-std,-std],[0,1],'--')
per = np.percentile(data, 95)
plt.plot([per,per],[0,1],'--')




############################
plt.plot(y[1][:-1],y[0]/np.max(y[0]), c='cyan')

#
mu, sigma = scipy.stats.norm.fit(data)
best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
best_fit_line = best_fit_line/np.max(best_fit_line)
argmax1 = np.argmax(y[0])
argmax2 = np.argmax(best_fit_line)
shift = argmax1-argmax2
plt.plot(bins, np.roll(best_fit_line,shift), c='red')
#plt.semilogy()
plt.ylim(bottom=0.001)
plt.show()


[938.6306  896.4506  888.8514  ... 894.6404  878.8259  857.12463]


In [57]:
     self.F (fluorescence):  (2131, 55740)
         self.Fneu (neuropile):  (2131, 55740)
         self.iscell (cell classifier output):  (2131, 2)
         self.ops:  ()
         self.spks (deconnvoved spikes):  (2131, 55740)
         self.stat (footprints?):  (2131,)
Number of cells:  (1316, 55740)
    
    
data = np.load('/media/cat/4TB/donato/steffen/DON-004366/20210228/suite2p/plane0/iscell.npy')
print (data.shape)


(2, 1788)


In [3]:
####################################################################
####################################################################
####################################################################

import csv

fname = '/media/cat/4TB/donato/laurenz/DON-005099_20210609_TR-BSL_S1-ACQ_DLC.csv'

def load_csv(fname):

    from numpy import genfromtxt
    locs = genfromtxt(fname, delimiter=',', dtype='str')

    print (locs.shape)
    print (locs)
    
    return locs


locs = load_csv(fname)
snout = np.float32(locs[3:,1:3])
print (snout.shape)




(36003, 43)
[['scorer' 'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000'
  'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000' ...
  'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000'
  'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000'
  'DLC_resnet50_open_arena_white_floorSep8shuffle1_200000']
 ['bodyparts' 'snout' 'snout' ... 'tail_tip' 'tail_tip' 'tail_tip']
 ['coords' 'x' 'y' ... 'x' 'y' 'likelihood']
 ...
 ['35997' '1929.4710693359375' '1526.892822265625' ... '1547.64892578125'
  '1165.356689453125' '0.9995512366294861']
 ['35998' '1929.422607421875' '1524.6185302734375' ...
  '1547.0325927734375' '1169.79541015625' '0.999924898147583']
 ['35999' '1929.53369140625' '1525.818603515625' ... '1547.0889892578125'
  '1175.59375' '0.9989499449729919']]
(36000, 2)


In [42]:
####################################################################
####################################################################
####################################################################
fname = '/media/cat/4TB/donato/laurenz/DON-005099_20210609_TRACES_ACQ.csv'

def load_traces(fname):
    
    from tqdm import tqdm, trange
    
    from numpy import genfromtxt
    data = genfromtxt(fname, delimiter=',', dtype='str')

    cells = np.float32(data[2:,1:]).T
    times = np.float32(data[2:,0])
    
    
    print ("F cells: ", cells.shape, "times: ", times.shape)
    return cells, times

F, F_times = load_traces(fname)


F cells:  (203, 36026) times:  (36026,)


In [43]:
# 

def find_nearest_times(times1, times2):
    
    idx3 = []
    for k in range(times2.shape[0]):
        spk = times2[k]
        time = np.argmin(np.abs(times1-spk))
        idx3.append(time)
        
    return idx3
    

def load_spks(fname, F, F_times):
    
    from tqdm import tqdm, trange
    
    from numpy import genfromtxt
    data = genfromtxt(fname, delimiter=',', dtype='str')
    print (data)
    
    # 
    times = np.float32(data[1:,0])
    cells = data[1:,1]
    probs = np.float32(data[1:,2])
    
    #
    cell_ids = np.sort(np.unique(cells))
    
    
    spks = np.zeros((F.shape[0], F.shape[1]),'float32')
    ctr=0
    for cell_id in tqdm(cell_ids):

        # find cell indexes
        idx = np.where(cells==cell_id)[0]
    
        # grab probabilities at indexes
        temp_probs = probs[idx]
    
        # get cell times
        temp_time = times[idx]
        
        # convert cell times to 
        times_idx = find_nearest_times(F_times, temp_time)
        
        # insert probabilities
        spks[ctr,times_idx] = temp_probs
        
        #
        ctr+=1       
    
    return spks

#
fname = '/media/cat/4TB/donato/laurenz/DON-005099_20210609_DECSPK_ACQ.csv'
spks = load_spks(fname, F, F_times)
print (spks.shape)



[['Time (s)' ' Cell Name' ' Value']
 ['0.049961' ' C000' ' 0.713386']
 ['1.149103' ' C000' ' 0.4412095']
 ...
 ['1799.295454' ' C202' ' 3.850622']
 ['1799.445337' ' C202' ' 0.6576414']
 ['1799.59522' ' C202' ' 4.768372e-07']]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [01:20<00:00,  2.53it/s]

(203, 36026) [0.         0.00693464 0.1320609  0.1387267  0.1695642  0.1888984
 0.2106581  0.2265217  0.2395024  0.2434065  0.2796097  0.3222035
 0.3739147  0.3852084  0.4074621  0.4396152  0.4541229  0.458006
 0.4976826  0.51337    0.52635    0.5421739  0.5680515  0.6075989
 0.6173973  0.6930106  0.6932306  0.6969218  0.713386   0.7638311
 0.7791519  0.7878394  0.8276944  0.8984568  0.9195459  0.9230608
 0.9605875  0.9806085  1.008928   1.018462   1.112808   1.156048
 1.158649   1.320862   1.532392   1.534408   1.68098    1.769603
 1.769874   2.134558   2.252327   2.400465   2.607677   2.639043
 2.705891   2.829987   3.891121   4.240234   4.389292   5.944993
 7.084787   7.231406   9.448754  ]


In [46]:
np.save('/media/cat/4TB/donato/laurenz/F.npy',F)
np.save('/media/cat/4TB/donato/laurenz/spks.npy',F)


In [45]:
#######################################################
#######################################################
#######################################################
plt.plot(spks[0], label='oasis spikes')
plt.plot(F[0], label='raw fluorescence')
plt.legend()
plt.show()

In [38]:
unit=0
idx = np.where(spks[:,1]==unit)[0]
print (idx.shape)
print (spks[idx,0].shape)

print (times.shape)

(6545,)
(6545,)
(36026,)


(1290907, 3) [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167.
 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180. 181.
 182. 183. 184. 185. 186. 187. 188. 189. 190. 191. 192. 193. 194

In [63]:
#############################################
####### COMPARE STEFFEN TO CAT METHOD #######
#############################################

from scipy.signal import butter, lfilter, freqz, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

#
def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y



iscell = np.load('/media/cat/4TB/donato/DON-006084/20210520/suite2p/plane0/iscell.npy')
idx = np.where(iscell[:,0])[0]

#
oasis = np.load('/media/cat/4TB/donato/DON-006084/20210520/suite2p/plane0/spks.npy')[idx]
print (oasis.shape)

#
F = np.load('/media/cat/4TB/donato/DON-006084/20210520/suite2p/plane0/F.npy')
F = F[idx]
print (F.shape)

steffen = np.load('/media/cat/4TB/donato/DON-006084/20210520/suite2p/plane0/20210520_steffen_binary.npy').T
steffen = steffen[idx]
print (steffen.shape)


cat = np.load('/media/cat/4TB/donato/DON-006084/20210520/suite2p/plane0/binarized_traces.npz',allow_pickle=True)
peaks = cat['binarized_peak']
print (peaks.shape)

cell = 500
scale = 100
fig=plt.figure()
c.high_cutoff = 1
t = np.arange(steffen.shape[1])/30.
f = F[cell]-np.mean(F[cell])
plt.plot(t, f,label='fluorescence', c='cyan')
plt.plot(t, butter_lowpass_filter(f, 1, 30), label='fluorescence - filtered', c='cyan',
        linewidth=2)

#
plt.plot(t, oasis[cell],label='oasis', c='white')
plt.plot(t, butter_lowpass_filter(oasis[cell], 1, 30)*10,label='oasis-filtered', c='white',
        linewidth=2)

#
plt.plot(t,steffen[cell]*scale*1.1, label='steffen', c='yellow', linewidth=2)
plt.plot(t,peaks[cell]*scale, label='cat-peaks', c='pink', linewidth=2)
plt.legend()
plt.show()


(584, 55740)
(584, 55740)
(584, 55740)
(584, 55740)
